<a href="https://colab.research.google.com/github/cedro3/BERT/blob/master/BERT_pretrained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日本語事前学習済みBERTで色々試してみる

# セットアップ

In [ ]:
# モジュールインストール
!pip install pyknp
!pip install transformers

In [ ]:
# JUMMAN++ インストール（5分ほどかかります)
!wget https://github.com/ku-nlp/jumanpp/releases/download/v2.0.0-rc2/jumanpp-2.0.0-rc2.tar.xz
!tar xfv jumanpp-2.0.0-rc2.tar.xz  
%cd jumanpp-2.0.0-rc2
!mkdir bld
%cd bld
!cmake .. -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=/usr/local
!make install -j2

In [ ]:
# 日本語学習済みBERTモデルをダウンロード　（10分ほどかかります）
import urllib.request
import sys
import os
import zipfile

# ディレクトリー作成
os.chdir('/content')
os.makedirs('bert', exist_ok = True)

# zipファイルダウンロード
url = 'http://nlp.ist.i.kyoto-u.ac.jp/DLcounter/lime.cgi?down=http://nlp.ist.i.kyoto-u.ac.jp/nl-resource/JapaneseBertPretrainedModel/Japanese_L-12_H-768_A-12_E-30_BPE_transformers.zip&name=Japanese_L-12_H-768_A-12_E-30_BPE_transformers.zip'
title = './bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers.zip'
urllib.request.urlretrieve(url,"{0}".format(title))

# zipファイル解凍
zipf = zipfile.ZipFile('./bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers.zip')
zipf.extractall('./bert/')
zipf.close()


# コード本体

In [ ]:
import torch
from transformers import BertTokenizer, BertForMaskedLM, BertConfig
import numpy as np
import textwrap
config = BertConfig.from_json_file('./bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/config.json')
model = BertForMaskedLM.from_pretrained('./bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/pytorch_model.bin', config=config)
bert_tokenizer = BertTokenizer('./bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/vocab.txt',
 do_lower_case=False, do_basic_tokenize=False)
from pyknp import Juman
jumanpp = Juman()


In [ ]:
# テキストへ[CLS],[SEP],[MASK]を追加する関数
def preparation(tokenized_text):
    
    # [CLS],[SEP]の挿入
    tokenized_text.insert(0, '[CLS]')  # センテンスの先頭に[CLS]を付ける
    tokenized_text.append('[SEP]')  # センテンスの最後に[SEP]を付ける 
        
    maru = []
    for i, word in enumerate(tokenized_text):
        if word =='。' and i !=len(tokenized_text)-2:  # 「。」の位置検出
            maru.append(i)

    for i, loc in enumerate(maru):
        tokenized_text.insert(loc+1+i, '[SEP]')  # 文の「。」の次に[SEP]を挿入する
        
    # □を[MASK]に置き換え　
    mask_index = []
    for index, word in enumerate(tokenized_text):
        if word =='□':  # 「□」の位置検出
            tokenized_text[index] = '[MASK]'
            mask_index.append(index)
    
    return tokenized_text, mask_index  

# BERTはセンター試験を解けるか？

In [ ]:
# テキストをIDテンソルに変換
text = "ギリシア人ポリュビオスは，著書『歴史』の中で，ローマ共和政の国制（政治体制）を優れたものと評価している。彼によれば，その国制には，コンスルという王制的要素，元老院という□制的要素，民衆という民主制的要素が存在しており，これら三者が互いに協調や牽制をしあって均衡しているというのである。ローマ人はこの政治体制を誇りとしており，それは，彼らが自らの国家を指して呼んだ「ローマの元老院と民衆」という名称からも読み取ることができる。共和政期末の内戦を勝ち抜いたかに見えた□でさえも，この体制を壊そうとしているという疑いをかけれ，暗殺されてしまった。"
result = jumanpp.analysis(text)  # 分かち書き
tokenized_text = [mrph.midasi for mrph in result.mrph_list()]  # 単語リストに変換
tokenized_text, mask_index = preparation(tokenized_text)  # [CLS],[SEP],[MASK]の追加
tokens = bert_tokenizer.convert_tokens_to_ids(tokenized_text)  # IDリストに変換
tokens_tensor = torch.tensor([tokens])  # IDテンソルに変換

In [ ]:
# [MASK]箇所を推論(TOP5)
model.eval() 
tokens_tensor = tokens_tensor.to('cuda')
model.to('cuda')
print(textwrap.fill(text, 45))
print()

with torch.no_grad():
  outputs = model(tokens_tensor)
  predictions = outputs[0]
  
  for i in range(len(mask_index)):
     _, predicted_indexes = torch.topk(predictions[0, mask_index[i]], k=5)
     predicted_tokens = bert_tokenizer.convert_ids_to_tokens(predicted_indexes.tolist())
     print(i, predicted_tokens)

# BERTによる文生成

In [ ]:
# 形態素解析
text = "我々が10年以内に月に行こうなどと決めたのは、それが容易だからではありません。むしろ困難だからです。この目標が、我々のもつ行動力や技術の最善といえるものを集結しそれがどれほどのものかを知るのに役立つこととなるからです。その挑戦こそ、我々が受けて立つことを望み、先延ばしすることを望まないものだからです。そして、これこそが、我々が勝ち取ろうと志すものであり、我々以外にとってもそうだからです。"
result = jumanpp.analysis(text)  # 分かち書き
tokenized_text = [mrph.midasi for mrph in result.mrph_list()]  # 単語リストに変換
tokenized_text, mask_index = preparation(tokenized_text)  # [CLS],[SEP]の追加
tokens = bert_tokenizer.convert_tokens_to_ids(tokenized_text)  # IDリストに変換
tokens_tensor = torch.tensor([tokens])  # IDテンソルに変換

In [ ]:
# 1単語予測関数
def predict_one(tokens_tensor, mask_index):

    model.eval()    
    tokens_tensor = tokens_tensor.to('cuda')
    model.to('cuda')
 
    with torch.no_grad():
      outputs = model(tokens_tensor)
      predictions = outputs[0]
 
      _, predicted_indexes = torch.topk(predictions[0, mask_index], k=5)
      predicted_tokens = bert_tokenizer.convert_ids_to_tokens(predicted_indexes.tolist())
    return predicted_tokens, predicted_indexes.tolist()

In [ ]:
# 文生成
for i in range(1,len(tokens_tensor[0])):
    tmp = torch.tensor(tokens_tensor)  # tokens_tensorをtmpにコピー
    tmp[0, i]=4  # i番目を[mask]に書き換え
    predicted_tokens, predicted_indexes =predict_one(tmp, i)  # [mask]を予測
    if predicted_indexes !=1:  # 予測が[UNK]でなければ
      tokens_tensor[0, i] = predicted_indexes[0]  # 予測IDの[0]番目でtokens_tensorのi番目を上書きする

target_list = tokens_tensor.tolist()[0]  
predict_list = bert_tokenizer.convert_ids_to_tokens(target_list)  
predict_sentence = ''.join(predict_list[1:])

print('------ original_text -------')
print(textwrap.fill(text,45))
print('------ predict_text -------')
print(textwrap.fill(predict_sentence,45))  